In [0]:
# Partial Bayesian Neural Networks
# Cyrus Maz & Nathan Friedman 2019
# Builds on code written by David Duvenaud


import matplotlib.pyplot as plt
import autograd.scipy.stats.norm as norm

import autograd.numpy as np
import autograd.numpy.random as npr
from autograd import grad
from autograd.misc.optimizers import adam
import os
import seaborn as sns

sns.set_style("white")



def shapes_and_num(layer_sizes):
    shapes = list(zip(layer_sizes[:-1], layer_sizes[1:]))
    N_weights = sum((m + 1) * n for m, n in shapes)
    return shapes, N_weights


def unpack_layers(weights, layer_sizes):
    """ unpacks weights [ns, nw] into each layers relevant tensor shape"""
    shapes, _ = shapes_and_num(layer_sizes)
    n_samples = len(weights)
    for m, n in shapes:
        yield weights[:, :m * n].reshape((n_samples, m, n)), \
              weights[:, m * n:m * n + n].reshape((n_samples, 1, n))
        weights = weights[:, (m + 1) * n:]


def reshape_weights(weights, layer_sizes):
    return list(unpack_layers(weights, layer_sizes))


def bnn_predict(weights, inputs, layer_sizes, activiation_fcn):
    if len(inputs.shape)<3: inputs = np.expand_dims(inputs, 0)  # [1,N,D]
    weights = reshape_weights(weights, layer_sizes)
    for W, b in weights:
        #print(W.shape, inputs.shape)
        outputs = np.einsum('mnd,mdo->mno', inputs, W) + b
        inputs = activiation_fcn(outputs)
    return outputs


def diag_gaussian_log_density(x, mu, log_std):
    return np.sum(norm.logpdf(x, mu, np.exp(log_std)), axis=0)  # [ns]

def logqu(x, mu, std):
    x = -np.sum(np.log(std)) - np.matmul(np.matmul(x-mu,np.diag(np.reciprocal(std**2))),x-mu)
    
    return -(len(mu)/2.)*np.log(2*np.pi) + x


def sample_weights(params, N_samples):
    mean, log_std = params
    
    exp_vector = np.array([0 if np.isnan(x) == True else np.exp(x) for x in log_std])
    
    output=rs.randn(N_samples, mean.shape[0]) * exp_vector + mean  # [ns, nw]
    
    return output

def sample_bnn(params, x, N_samples, layer_sizes, act):
    bnn_weights = sample_weights(params, N_samples)
    
    f_bnn = bnn_predict(bnn_weights, x, layer_sizes, act)[:, :, 0]
    return f_bnn # [ns, nd]


def log_pdf_prior(weights, prior_params, sd):
    if prior_params is None:
        return diag_gaussian_log_density(weights, 0, np.log(sd))
    else:
        prior_mean, prior_log_std = prior_params
        return diag_gaussian_log_density(weights, prior_mean, prior_log_std)

def log_pdf_prior2(weights, prior_params, sd):
    if prior_params == None:
        d = len(weights)
        x = -(d/2)*np.log(2*np.pi)- d*np.log(sd)- np.sum(weights**2)/(2*sd^2)
    return x
    

def log_like(y, f, sd):
    d = np.shape(f)[0]*np.shape(f)[1]
    a = d*np.log(sd)
    c = (d/2.)*np.log(2*np.pi)
    SS = np.sum((f - y)**2, axis=1) / (2*sd**2)
    return - c - a - SS  # [ns]



def vlb_objective(params, x, y, layer_sizes, n_samples, 
                  prior_sd=10, model_sd=0.1, prior_params=None, act=np.tanh):
    Pweight = np.argwhere(np.isnan(params[1]) == False)
    Fweight = np.argwhere(np.isnan(params[1]) == True)
    mean, log_std = params
    log_std = log_std[Pweight]
    
    
    
    weights = sample_weights(params, n_samples)
    
    if (len(Pweight)==0): 
      logq=[0]
      log_prior=0
    else: 
      logq = np.array([logqu(weights[k,Pweight], mu = params[0][Pweight], std = np.exp(params[1][Pweight])) for k in range(n_samples)])
      log_prior = np.array([log_pdf_prior2(weights[k,Pweight], prior_params, prior_sd) for k in range(n_samples)])    
  
    f_bnn = bnn_predict(weights, x, layer_sizes, act)[:, :, 0]
   
    log_likelihood = log_like(y.T, f_bnn, model_sd)
   
    retval = np.mean(logq[0] -log_likelihood - log_prior)
    
    return retval



def init_var_params(layer_sizes, Pweights,scale=-5, scale_mean=1):
    _, num_weights = shapes_and_num(layer_sizes)
    X = rs.randn(num_weights)*scale_mean, np.ones(num_weights)*scale
    X[1][Pweights] = np.nan
    
    return  X # mean, log_std

## Build data sets:
def build_dataset_1(n_data=50, noise_std=0.1):
    rs = npr.RandomState(0)
    inputs = np.linspace(-15, 15, num=n_data)

    targets = inputs + rs.randn(n_data) * noise_std
    inputs = inputs

    return inputs[:,None], targets[:,None]


def build_dataset_2(n_data=50, noise_std=0.1):
    rs = npr.RandomState(0)
    inputs = np.linspace(-15, 15, num=n_data)

    targets = inputs**2 /10 + rs.randn(n_data) * noise_std
    inputs = inputs

    return inputs[:,None], targets[:,None]

def build_dataset_3(n_data=50, noise_std=0.1):
    rs = npr.RandomState(0)
    inputs = np.linspace(-15, 15, num=n_data)

    targets = 3*np.sin(inputs)+inputs + rs.randn(n_data) * noise_std
    inputs = inputs

    return inputs[:,None], targets[:,None]


def build_dataset_4(n_data=50, noise_std=0.1):
    rs = npr.RandomState(0)
    inputs = np.linspace(-15, 15, num=n_data)

    targets = np.log(inputs**2 )*3 + rs.randn(n_data) * noise_std
    inputs = inputs

    return inputs[:,None], targets[:,None]

def build_dataset_5(n_data=50, noise_std=0.1):
    rs = npr.RandomState(0)
    inputs = np.linspace(-15, 15, num=n_data)

    targets = 6*np.sin(inputs**2 /15)**2+ 6*np.cos(inputs**2 /15)+rs.randn(n_data) * noise_std
    inputs = inputs

    return inputs[:,None], targets[:,None]

def build_dataset_6(n_data=50, noise_std=0.1):
    rs = npr.RandomState(0)
    # inputs = np.linspace(-n_data/2+10, n_data/2+10, num=n_data)
    inputs = np.linspace(-15, 15, num=n_data)
    targets = (inputs**2)/10 * np.sin(inputs**2 / 20 + inputs) + inputs/20 + np.cos(inputs)+ rs.randn(n_data) * noise_std


    return inputs[:,None], targets[:,None]


def plot_mean_std(x_plot, p, D, path):
    x, y = D
    # col = colors[plot]
    col = sns.xkcd_rgb["watermelon"]

    mean, std = np.mean(p, axis=1), np.std(p, axis=1)

    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(x_plot, mean, lw=2)
    ax.fill_between(x_plot, mean - 1.96 * std, mean + 1.96 * std, color=col)  # 95% CI
    ax.plot(x, y, 'ko', ms=4)
    # ax.tick_params(labelleft='off', labelbottom='off')
    ax.set_ylim([y_lower, y_upper])
    ax.set_xlim([x_lower, x_upper])
    plt.savefig(path, bbox_inches='tight',dpi = 300)
    plt.close() 

def plot_samples(x_plot, p, D, path):
    x, y = D

    fig = plt.figure(facecolor='white')
    ax = fig.add_subplot(111)
    ax.plot(x_plot, p, lw=1)
    ax.plot(x, y, 'ko', ms=4)
    # ax.tick_params(labelleft='off', labelbottom='off')
    ax.set_ylim([y_lower, y_upper])
    ax.set_xlim([x_lower, x_upper])
    
    plt.savefig(path, bbox_inches='tight',dpi = 300)
    plt.close()     


    
rbf = lambda x: np.exp(-x ** 2)

def objective(params,t):
    return vlb_objective(params, inputs, targets, arch, n_samples=10, act=rbf)
        
def callback_func(params,t,g):
    Obj_count[t] = -objective(params, t)
    if t % 200 == 0:
        print("ITER {} | OBJ {}".format(t, -objective(params, t)))        

def RandomSam(arch,k):
    Tweights = sum([arch[x]*arch[x+1] + arch[x+1] for x in range(len(arch)-1)])
    return(rs.choice(range(Tweights),k, replace = False))

def functions(x,numfun):
    if numfun == 1:
        y = x
    elif numfun == 2:
        y = x**2 /10
    elif numfun == 3:
        y = 3*np.sin(x) + x
    elif numfun == 4:
        y = 3*np.log(x**2)
    elif numfun == 5:
        y = 6*np.sin(x**2 /15)**2+ 6*np.cos(x**2 /15)
    elif numfun == 6:
        y = (1./10.)* x**2 * np.sin(x**2 /20 + x) + x/20 + np.cos(x)
    return(y)



In [3]:
# authorize google colab to access Google Drive for saving plots
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
file_directory='gdrive/My Drive/CSC2516_project/plots_15/'

# Training data parameters
noise_std=0.5
n_data=40

# Adam Parameters
step_size=0.005
num_iters=601

# NN Parameters
N_samples = 10
arch = [1, 40, 40, 1]

# Plot Parameters
y_lower=-30
y_upper=30
x_lower=-30
x_upper=30
num_xs = 600

# Training Parameters
numper = [1,10,6,3,1]
kvals = [0,180,440,880,1761]


In [0]:
if not os.path.exists(file_directory): os.makedirs(file_directory);

plot_inputs = np.linspace(x_lower, x_upper, num=num_xs)  

# set seed for reproducability   
rs = npr.RandomState(2516)

# Loop:
for n in range(1,6):

    if n==1: inputs, targets = build_dataset_1(n_data,noise_std);
    if n==2: inputs, targets = build_dataset_2(n_data,noise_std);
    if n==3: inputs, targets = build_dataset_3(n_data,noise_std);
    if n==4: inputs, targets = build_dataset_4(n_data,noise_std);
    if n==5: inputs, targets = build_dataset_5(n_data,noise_std); 
    if n==6: inputs, targets = build_dataset_6(n_data,noise_std); num_iters=1001

    Preds = np.zeros((len(kvals),num_xs))

    for l in range(len(kvals)):

        Temp_hold = np.zeros((numper[l],num_xs))

        for j in range(numper[l]):
          
            print("*** Ensemble of " + str(numper[l]) + " networks;")    
            print("*** " + str(round(kvals[l]/1761 ,2)*100) + "% bayesian;")
            print("*** function " +str(n))
            print("*** network " + str(j+1) + " of " + str(numper[l]) + ";")
            
            k = kvals[l]
            r  = 1761 - k
            Pweights = RandomSam(arch,r)

            init_var_param = init_var_params(arch,Pweights)


            Obj_count = np.zeros(num_iters)
            var_params = adam(grad(objective), init_var_param,
                      step_size=step_size, num_iters=num_iters, callback=callback_func)

            # Plot learning curve:
            learning_curve_plot_name = "f"+str(n)+"_learn_curve_k_"+str(kvals[l])+"_nn_"+str(j+1)+"_of_"+str(numper[l])+".png"
            learning_curve_plot_path = file_directory + learning_curve_plot_name
            plt.plot(Obj_count)
            plt.xlabel("Iteration")
            plt.ylabel("Objective")
            plt.savefig(learning_curve_plot_path, bbox_inches='tight',dpi = 300)
            plt.close() 
            
            print("Learning Curve plot done: "+learning_curve_plot_path)

            f_bnn = sample_bnn(var_params, plot_inputs[:, None], N_samples, arch, rbf)
            yhat = np.mean(f_bnn,axis = 0) 
            Temp_hold[j,] = yhat
            
            
            # Plot CI's
            CI_plot_name= "f"+str(n)+"_CI_k"+str(kvals[l])+"_nn_"+str(j+1)+"_of_"+str(numper[l])+".png"
            CI_plot_path= file_directory+CI_plot_name
            
            samples_plot_name= "f"+str(n)+"_samples_k"+str(kvals[l])+"_nn_"+str(j+1)+"_of_"+str(numper[l])+".png"
            samples_plot_path= file_directory+samples_plot_name
            
#             sns.set_style("white")
            
            D=inputs,targets
            
            f_bnn=np.transpose(f_bnn)
            plot_mean_std(plot_inputs, f_bnn, D, path=CI_plot_path) ####
            print("CI plot done: "+ CI_plot_path)
            
            plot_samples(plot_inputs, f_bnn, D, path=samples_plot_path) ####
            print("Sample predictions plot done: "+samples_plot_path)            
            
            



        Preds[l,] = np.mean(Temp_hold,axis = 0)


    # Plot Regressions 
    regression_plot_name="function"+str(n)+".png"
    regression_plot_path=file_directory+regression_plot_name

    y_act = functions(plot_inputs,n) 
    To_plot = np.c_[y_act,np.transpose(Preds)]

    plot_labs = ['Observations','y(x)','Non-Bayesian', '10% Bayesian', '25% Bayesian','50% Bayesian' ,'Full Bayesian']
    plt.plot(inputs,targets, "o", alpha = 0.2, color = 'black')
    plt.plot(plot_inputs,To_plot)
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend(plot_labs,loc='best')
    plt.savefig(regression_plot_path, bbox_inches='tight',dpi = 300)
    plt.close() 
    print("Regression Plot done: "+ regression_plot_path)

    

*** Ensemble of 1 networks;
*** 0.0% bayesian;
*** function 5
*** network 1 of 1;
ITER 0 | OBJ -64884.51401482076


/usr/local/lib/python3.6/dist-packages/autograd/numpy/numpy_vjps.py:444: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return lambda g: g[idxs]


ITER 200 | OBJ -14467.83868628395
ITER 400 | OBJ -9582.660878687404
ITER 600 | OBJ -4406.729823895868
Learning Curve plot done: gdrive/My Drive/CSC2516_project/plots_15/f5_learn_curve_k_0_nn_1_of_1.png
CI plot done: gdrive/My Drive/CSC2516_project/plots_15/f5_CI_k0_nn_1_of_1.png
Sample predictions plot done: gdrive/My Drive/CSC2516_project/plots_15/f5_samples_k0_nn_1_of_1.png
*** Ensemble of 10 networks;
*** 10.0% bayesian;
*** function 5
*** network 1 of 10;
ITER 0 | OBJ -87670.73632442483
ITER 200 | OBJ -14640.608011833325
ITER 400 | OBJ -10706.746849212246
ITER 600 | OBJ -7259.2502020788215
Learning Curve plot done: gdrive/My Drive/CSC2516_project/plots_15/f5_learn_curve_k_180_nn_1_of_10.png
CI plot done: gdrive/My Drive/CSC2516_project/plots_15/f5_CI_k180_nn_1_of_10.png
Sample predictions plot done: gdrive/My Drive/CSC2516_project/plots_15/f5_samples_k180_nn_1_of_10.png
*** Ensemble of 10 networks;
*** 10.0% bayesian;
*** function 5
*** network 2 of 10;
ITER 0 | OBJ -248735.6441895

[5, 6]